In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# import data sets

path = r'/Users/lennartzeidler/Desktop/Uni/Instacart Basket Analysis 08.10.2022 (4)'

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_sale_performance.pkl'))

In [3]:
df = ords_prods_merge[:1000000]

In [4]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_range,busiest_day,busiest_days,busiest_hours
0,2539329,1,1,2,8,NaN,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,Regulary busy days,Regulary busy hours
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Mid-range product,Regularly busy,Slowest days,Regulary busy hours
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Mid-range product,Regularly busy,Slowest days,Busiest hours
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Slowest days,Regulary busy hours
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Slowest days,Busiest hours


In [12]:
# first step goup data

df.groupby('department_id').agg({'order_number': ['mean']}).shape

(8, 1)

In [ ]:
df['department_id']

### Aggregation Data with tzransform() ==> customer loyalty

In [6]:
# create new column containing max order number o

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/zw/37lckzg54ll7byfmgjl5pkw40000gn/T/ipykernel_11827/1197110535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
df.head(100)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_range,busiest_day,busiest_days,busiest_hours,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,Regulary busy days,Regulary busy hours,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Mid-range product,Regularly busy,Slowest days,Regulary busy hours,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Mid-range product,Regularly busy,Slowest days,Busiest hours,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Slowest days,Regulary busy hours,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Slowest days,Busiest hours,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,1,5,12,NaN,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,Regulary busy days,Busiest hours,1
96,1469869,377,3,5,17,3.0,196,9,0,Soda,7,9.0,Mid-range product,Regularly busy,Regulary busy days,Regulary busy hours,3
97,1927023,387,2,4,10,22.0,196,3,0,Soda,7,9.0,Mid-range product,Least busy,Slowest days,Busiest hours,2
98,858092,420,4,1,19,30.0,196,2,0,Soda,7,9.0,Mid-range product,Regularly busy,Busiest days,Regulary busy hours,5


In [8]:
# turning max row display off
# pd.options.display.max_rows = None

In [9]:
# creating loyalty flag

df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

/var/folders/zw/37lckzg54ll7byfmgjl5pkw40000gn/T/ipykernel_11827/2257354214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [10]:
df['loyalty_flag'].value_counts(dropna = False)

Regular customer    463844
Loyal customer      335086
New customer        201070
Name: loyalty_flag, dtype: int64

In [11]:
df[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10
